# ipl I317B Sécurité : labos
## Semaine 1 : introduction et outils python

### Partie 1 : manipuler du texte

#### Exercice 1
Pour ce premier exercice, nous allons manipuler un court extrait de logs d'un serveur web nginx.  
Pour chaque ligne de celui-ci, extrayez:
- l'adresse IP
- le timestamp
- le chemin demandé

In [1]:
logs = """
    91.160.48.68 - - [3/Sep/2020:18:18:10 +0200] "GET /?C=M;O=D HTTP/1.1" 301 178 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0" "-"
    78.224.72.158 - - [3/Sep/2020:18:24:52 +0200] "GET / HTTP/1.1" 301 178 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0" "-"
    192.241.199.4 - - [3/Sep/2020:19:11:15 +0200] "GET / HTTP/1.1" 301 178 "-" "Mozilla/5.0 zgrab/0.x" "-"
    40.121.69.161 - - [3/Sep/2020:19:16:32 +0200] "GET /.env HTTP/1.1" 301 178 "-" "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36" "-"
    40.121.69.161 - - [3/Sep/2020:19:16:32 +0200] "POST / HTTP/1.1" 301 178 "-" "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36" "-"
    185.172.110.223 - - [3/Sep/2020:19:16:37 +0200] "GET / HTTP/1.1" 301 178 "-" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:76.0) Gecko/20100101 Firefox/76.0" "-"
    66.249.75.16 - - [3/Sep/2020:19:23:28 +0200] "GET / HTTP/1.1" 301 178 "-" "Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.97 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)" "-"
    66.249.75.23 - - [3/Sep/2020:19:23:30 +0200] "GET / HTTP/1.1" 200 0 "-" "
"""

In [2]:
for line in logs.split("\n"):
      if line:
            print("IP:", line.split(" ")[0],
                    "- Timestamp:", line.split("[")[1].split("]")[0],
                    "- Path:", line.split("\"")[1].split(" ")[1]
            )


IP:  - Timestamp: 3/Sep/2020:18:18:10 +0200 - Path: /?C=M;O=D
IP:  - Timestamp: 3/Sep/2020:18:24:52 +0200 - Path: /
IP:  - Timestamp: 3/Sep/2020:19:11:15 +0200 - Path: /
IP:  - Timestamp: 3/Sep/2020:19:16:32 +0200 - Path: /.env
IP:  - Timestamp: 3/Sep/2020:19:16:32 +0200 - Path: /
IP:  - Timestamp: 3/Sep/2020:19:16:37 +0200 - Path: /
IP:  - Timestamp: 3/Sep/2020:19:23:28 +0200 - Path: /
IP:  - Timestamp: 3/Sep/2020:19:23:30 +0200 - Path: /


#### Exercice 2
Avec les mêmes logs que l'exercice précédent, comptez et affichez les différents user-agent.

In [3]:
liste = {}

for line in logs.split("\n"):
    if line:
        userAgent = line.split("\"")[5]
        
        if userAgent not in liste:
            liste[userAgent] = 1
        else:
            liste[userAgent] += 1

for key, value in liste.items():
    print(value, ":", key)

2 : Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:80.0) Gecko/20100101 Firefox/80.0
1 : Mozilla/5.0 zgrab/0.x
2 : Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.129 Safari/537.36
1 : Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:76.0) Gecko/20100101 Firefox/76.0
1 : Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.97 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)
1 : 


### Partie 2 : requests et BeautifulSoup

Assurez vous que les paquets suivant soient bien installé :

`pip3 install --user requests bs4`

`sudo pip3 install requests bs4`

Depuis un jupyter notebook, vous pouvez probablement juste executer la case suivante :

In [ ]:
!pip3 install --user requests bs4

Le packet requests nous permet de faire des requêtes web facilement, par exemple :

In [3]:
import requests

ipl_req = requests.get("https://www.vinci.be/fr/formations/informatique-developpement-applications")

if ipl_req.ok:
    print("ok")
    ipl_html = ipl_req.text
else:
    print(ipl_req.status_code)
    print(ipl_req.reason)

# Doc requests : https://requests.readthedocs.io/en/latest/ )

ok


Nous pouvons ensuite lire le code source de la page en affichant `ipl_html`.

In [4]:
ipl_html[:500]  # 500 premier caractères

'\n\n\n\n    <!doctype html>\n<html class="no-js preload" lang="fr">\n<head>\n    \n<!-- Biskit SDK Script -->\n<script>\nwindow.biskitSdk = window.biskitSdk || {}\nwindow.biskitSdk.consentFunction = null\nwindow.biskitSdk.registerConsentFunction = (callback) => {\n    window.biskitSdk.consentFunction = callback\n    return true\n}\n</script>\n<!-- END Biskit SDK Script -->\n\n<!-- Google Tag Manager -->\n<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\nnew Date().getTime(),event:\'gtm.js\'});var f=d'

Parser du html avec des split ou des regex deviendrait vite infernal, c'est là qu'intervient la librairie BeautifulSoup qui nous permet de parcourir beaucoup plus simplement le dom notamment avec les fonctions `find` et `find_all`.

In [7]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(ipl_html)

# pour récupérer le contenu de la balise <title> :
title = soup.find("title").get_text()

print(title)

Informatique - développement d’applications | HE VINCI


La fonction `find` et la fonction `find_all` vous renverons un object de type `tag` (balise) sur lequel vous pouvez faire :
  * d'autres `find()` et `find_all()`
  * `get_text()` pour récupérer tout le contenu textuel de la balise
  * `["nom de l'attribut"]` pour récupérer un attribut en particulier (exemple: `tag['href']`)
  * ...

( Doc BeautifulSoup4 : https://www.crummy.com/software/BeautifulSoup/bs4/doc/ )

#### Exercice 1:
Récupérez le contenue de la page suivante et affichez son titre principal (la balise "h1") :https://www.vinci.be/fr/formations/informatique-developpement-applications

In [6]:
import requests
from bs4 import BeautifulSoup

ipl_req = requests.get("https://www.vinci.be/fr/formations/informatique-developpement-applications")

if ipl_req.ok:
    ipl_html = ipl_req.text

    soup = BeautifulSoup(ipl_html)

    title = soup.find("h1").get_text()

    print(title)
else:
    print(ipl_req.status_code)
    print(ipl_req.reason)

ok
Informatique - développement d’applications


---

soup.find_all("a", href="https://www.facebook.com/HEVINCI/")Il est possible de faire des requêtes beaucoup plus poussée en ajoutant des arguments correspondant aux attributs des balises que nous cherchons, par exemple pour trouver les balises `<div class="footer"></div>` :

```
soup.find_all("div", class_="footer")
# NB: class est un mot clé réservé en python, d'où l'ajout du "_" dans BeautifulSoup

# BeautifulSoup accepte également la notation suivante où class n'est plus un mot clé mais un string :
soup.find_all("div", {"class": "footer"})
```

Ou encore pour trouver tous les liens vers la page facebook de l'ipl :

`soup.find_all("a", href="https://www.facebook.com/HEVINCI/")`

BeautifoulSoup accepte également des regex compilée en argument, il est donc possible de récupérer la liste des url qui ne sont pas du domaines de "vinci.be" :

```
import re

not_vinci = re.compile("^http(s?)://(?!www.vinci.be/).*$")
soup.find_all("a", href=not_vinci)
```

Il est également possible d'utiliser une fonction arbitraire pour valider les liens. Par exemple, pour trouver tous les liens de moins de 30 caractères :

```
def less_than_30(tag):
    return len(tag) < 30  # on retourne un booleen
    
# on passe notre fonction en paramètre sans l'executer :
soup.find_all("a", href=less_than_30)
```

#### Exercice 2:

Utilisez des regex pour récupérer la liste des liens http (et donc pas https) sur la page :  
https://www.vinci.be/fr/formations/informatique-developpement-applications

In [14]:
import requests
import re
from bs4 import BeautifulSoup

http = re.compile("^http://*")

ipl_req = requests.get("https://www.vinci.be/fr/formations/informatique-developpement-applications")

if ipl_req.ok:
    ipl_html = ipl_req.text

    soup = BeautifulSoup(ipl_html)

    title = soup.findAll("a", href=http)

    print(title)
else:
    print(ipl_req.status_code)
    print(ipl_req.reason)

[<a href="http://www.enseignement.be/index.php?page=4323" rel="noreferrer noopener" target="_blank">enseignement.be</a>]


#### Exercice 3:

Utilisez une fonction pour récupérer tous les liens qui finissent avec une extension désignant un type de fichier (ex: ".html", ".php", ".pdf", ...) sur la page :  
https://www.vinci.be/fr/formations/informatique-developpement-applications

In [16]:
import requests
import re
from bs4 import BeautifulSoup

def get_links_with_extensions(url, extensions):
    response = requests.get(url)
    response.raise_for_status()
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    pattern = re.compile(r".*\.(" + "|".join(extensions) + r")$")
    
    links = soup.find_all('a', href=True)
    
    filtered_links = [link['href'] for link in links if pattern.match(link['href'])]
    
    return filtered_links

url = "https://www.vinci.be/fr/formations/informatique-developpement-applications"
extensions = ["html", "php", "pdf"]
filtered_links = get_links_with_extensions(url, extensions)

for link in filtered_links:
    print(link)

https://www.vinci.be/uploads/brochures/INFORMATIQUE_HEVINCI_23-24.pdf
https://progcours.vinci.be/cocoon/programmes/P1INFV01_C.html


#### Exercice 4:
Depuis le site du Centre Cybersecurité Belgique https://ccb.belgium.be/fr/actualit%C3%A9s , récupérez la liste des articles récents et affichez leurs titres ainsi qu'un lien vers l'article.

In [56]:
import requests
from bs4 import BeautifulSoup

url = "https://ccb.belgium.be/fr/actualités"

ipl_req = requests.get(url)

if ipl_req.ok:
    ipl_html = ipl_req.text

    soup = BeautifulSoup(ipl_html)

    title = soup.findAll("h2")
    articles = soup.findAll("article")

    for i in range(len(title)):
        print(title[i].get_text(), "\n  ↳ https://ccb.belgium.be" + articles[i].get("onclick").split("'")[1])

else:
    print(ipl_req.status_code)
    print(ipl_req.reason)

La Belgique parmi les meilleurs au monde pour la cybersécurité 
  ↳ https://ccb.belgium.be/fr/actualit%C3%A9/la-belgique-parmi-les-meilleurs-au-monde-pour-la-cybers%C3%A9curit%C3%A9
Evénement hybride SANS et CCB sur la sécurité du cloud 
  ↳ https://ccb.belgium.be/fr/actualit%C3%A9/ev%C3%A9nement-hybride-sans-et-ccb-sur-la-s%C3%A9curit%C3%A9-du-cloud
Venez nous rendre visite à BruCON 2024 
  ↳ https://ccb.belgium.be/fr/actualit%C3%A9/venez-nous-rendre-visite-%C3%A0-brucon-2024
Mesures administratives et amendes sous NIS2 
  ↳ https://ccb.belgium.be/fr/actualit%C3%A9/mesures-administratives-et-amendes-sous-nis2-0
Graves problèmes informatiques dans des entreprises belges qui appliquent la mise à jour de CrowdStrike 
  ↳ https://ccb.belgium.be/fr/actualit%C3%A9/graves-probl%C3%A8mes-informatiques-dans-des-entreprises-belges-qui-appliquent-la-mise-%C3%A0-jour
La notification des incidents NIS2 
  ↳ https://ccb.belgium.be/fr/actualit%C3%A9/la-notification-des-incidents-nis2
Les mesures de 

#### Exercice 5:
La lib requests permet également d'effectuer de requête POST. Pour tester cette fonctionnalitée, créez un "httpDump" sur le site https://httpdump.app/ puis utilisez requests pour poster des données vers votre requestbin.

> httpdump
> This free service gives you a unique URL to send requests to and inspect them. 


Voir : https://requests.readthedocs.io/en/master/user/quickstart/#make-a-request (ou la cheatsheet)

In [59]:
import requests
from bs4 import BeautifulSoup

url = "https://httpdump.app/dumps/89bc59bc-92ee-4a7d-8298-c18112b83598"

# post request
req = requests.post(url, data={"usename": "IweZix", "password": "MySuperPassword"})

if req.ok:
    print("ok")
else:
    print(req.status_code)
    print(req.reason)

ok
python-requests/2.32.2


#### Exercice 6:
Vous avez certainement constaté sur votre requestbin que la librairie requests utilise par défaut un User-Agent explicite "*je suis un lib python pour faire des requêtes*".  
Documentez-vous sur la manière de changer celui-ci dans requests et remplacez le dans votre précédente requête vers httpdump par quelque chose de plus discret, par exemple l'user-agent de la dernière version de chrome :  
`Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36`

source: https://www.whatismybrowser.com/guides/the-latest-user-agent/chrome

In [66]:
import requests
from bs4 import BeautifulSoup

url = "https://httpdump.app/dumps/89bc59bc-92ee-4a7d-8298-c18112b83598"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# post request
req = requests.post(url, data={"usename": "IweZix", "password": "MySuperPassword"}, headers=headers)

if req.ok:
    print("ok")
else:
    print(req.status_code)
    print(req.reason)

ok


# Partie 3 (bonus):

Note: dans les jupyter notebook, vous pouvez exécuter des commandes shell en préfixant une cellule d'un `!`. Par exemple : `!date`.

#### Exercice 1

Pour ce premier exercice, nous allons manipuler un court extrait de logs "logs.txt" d'un serveur web nginx (le même que pour la partie 1).

Pour chaque ligne de celui-ci, extrayez:

    l'adresse IP
    le timestamp
    le chemin demandé

En n'utilisant que des commandes bash : cat, grep, cut, sort, uniq,...

In [75]:
!cut -d ' ' -f1,4,7 logs.txt

91.160.48.68 [3/Sep/2020:18:18:10 /?C=M;O=D
78.224.72.158 [3/Sep/2020:18:24:52 /
192.241.199.4 [3/Sep/2020:19:11:15 /
40.121.69.161 [3/Sep/2020:19:16:32 /.env
40.121.69.161 [3/Sep/2020:19:16:32 /
185.172.110.223 [3/Sep/2020:19:16:37 /
66.249.75.16 [3/Sep/2020:19:23:28 /
66.249.75.23 [3/Sep/2020:19:23:30 /


In [76]:
!echo "TODO" | grep "DO"

TODO


#### Exercice 2
Avec les mêmes logs que l'exercice précédent, comptez et affichez les différents user-agent.

In [2]:
!echo "TODO" | grep "DO"

TODO
